In [ ]:
import os
import zipfile
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import cv2
import numpy as np


In [ ]:
# Step 1: Unzip the Dataset
zip_path = '/content/drive/MyDrive/computer vision/Face detection.v1-fddb.tensorflow.zip'
unzip_path = '/content/Face detection'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_path)


In [6]:
pip install openvino-dev


In [ ]:


# Step 2: Load and Prepare Data
def load_annotations(folder_path):
    annotations_file = os.path.join(folder_path, '_annotations.csv')
    annotations = pd.read_csv(annotations_file)
    data = []
    for _, row in annotations.iterrows():
        if row['class'] == 'face':
            image_path = os.path.join(folder_path, row['filename'])
            if os.path.exists(image_path):
                data.append({
                    'filename': image_path,
                    'bbox': [row['xmin'], row['ymin'], row['xmax'], row['ymax']],
                })
            else:
                print(f"Warning: Missing file - {image_path}")
    return data

train_data = load_annotations(os.path.join(unzip_path, 'train'))
valid_data = load_annotations(os.path.join(unzip_path, 'valid'))

# Step 3: Data Generator
def preprocess_image(image_path, bbox, target_size=(224, 224)):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Unable to read image {image_path}")
        return None, None
    img = cv2.resize(img, target_size)
    bbox = [coord / max(img.shape[:2]) for coord in bbox]  # Normalize bounding box
    return img, bbox

def data_generator(data, batch_size=32, target_size=(224, 224)):
    while True:
        batch_images, batch_bboxes = [], []
        for _ in range(batch_size):
            idx = np.random.randint(len(data))
            item = data[idx]
            img, bbox = preprocess_image(item['filename'], item['bbox'], target_size)
            if img is not None and bbox is not None:
                batch_images.append(img)
                batch_bboxes.append(bbox)
        yield np.array(batch_images) / 255.0, np.array(batch_bboxes)

# Step 4: Build Model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
outputs = Dense(4, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=outputs)

model.compile(optimizer=Adam(learning_rate=1e-4), loss='mean_squared_error')

# Step 5: Train Model
train_gen = data_generator(train_data)
valid_gen = data_generator(valid_data)

steps_per_epoch = len(train_data) // 32
validation_steps = len(valid_data) // 32

model.fit(
    train_gen,
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_gen,
    validation_steps=validation_steps,
    epochs=1,
)

# Save the model
model.save('/content/face_detection_model.h5')

print("Model training completed and saved at '/content/face_detection_model.h5'.")


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
1305/1305 ━━━━━━━━━━━━━━━━━━━━ 318s 200ms/step - loss: 0.7952 - val_loss: 0.8653


Model training completed and saved at '/content/face_detection_model.h5'.


In [ ]:
# Save the model
model.save('/content/drive/MyDrive/computer vision/face_detection_model.h5')

In [ ]:
!mo --input_model /content/face_detection_model.h5 --framework tf --output_dir /content/output


[ INFO ] MO command line tool is considered as the legacy conversion API as of OpenVINO 2023.2 release.
In 2025.0 MO command line tool and openvino.tools.mo.convert_model() will be removed. Please use OpenVINO Model Converter (OVC) or openvino.convert_model(). OVC represents a lightweight alternative of MO and provides simplified model conversion API. 
Find more information about transition from MO to OVC at https://docs.openvino.ai/2023.2/openvino_docs_OV_Converter_UG_prepare_model_convert_model_MO_OVC_transition.html
2025-01-10 03:31:09.983478: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-10 03:31:10.004613: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-10 03:31:10.010805: E external/local_xla/xla/s

In [2]:
import os
import zipfile
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import cv2
import numpy as np

# Step 1: Unzip the Dataset
zip_path = '/content/drive/MyDrive/computer vision/Face detection.v1-fddb.tensorflow.zip'
unzip_path = '/content/Face detection'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_path)

# Step 2: Load and Prepare Data
def load_annotations(folder_path):
    annotations_file = os.path.join(folder_path, '_annotations.csv')
    annotations = pd.read_csv(annotations_file)
    data = []
    for _, row in annotations.iterrows():
        if row['class'] == 'face':
            image_path = os.path.join(folder_path, row['filename'])
            if os.path.exists(image_path):
                data.append({
                    'filename': image_path,
                    'bbox': [row['xmin'], row['ymin'], row['xmax'], row['ymax']],
                })
            else:
                print(f"Warning: Missing file - {image_path}")
    return data

train_data = load_annotations(os.path.join(unzip_path, 'train'))
valid_data = load_annotations(os.path.join(unzip_path, 'valid'))

# Step 3: Data Generator
def preprocess_image(image_path, bbox, target_size=(224, 224)):
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Unable to read image {image_path}")
        return None, None
    img = cv2.resize(img, target_size)
    bbox = [coord / max(img.shape[:2]) for coord in bbox]  # Normalize bounding box
    return img, bbox

def data_generator(data, batch_size=32, target_size=(224, 224)):
    while True:
        batch_images, batch_bboxes = [], []
        for _ in range(batch_size):
            idx = np.random.randint(len(data))
            item = data[idx]
            img, bbox = preprocess_image(item['filename'], item['bbox'], target_size)
            if img is not None and bbox is not None:
                batch_images.append(img)
                batch_bboxes.append(bbox)
        yield np.array(batch_images) / 255.0, np.array(batch_bboxes)

# Step 4: Build Model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
outputs = Dense(4, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=outputs)

model.compile(optimizer=Adam(learning_rate=1e-4), loss='mean_squared_error')

# Step 5: Train Model
train_gen = data_generator(train_data)
valid_gen = data_generator(valid_data)

steps_per_epoch = len(train_data) // 32
validation_steps = len(valid_data) // 32

model.fit(
    train_gen,
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_gen,
    validation_steps=validation_steps,
    epochs=1,
)

# Save the model as a .pb file
tf.saved_model.save(model, "/content/saved_model")

# Convert to OpenVINO format
print("Converting the model to OpenVINO format...")
os.system(f"""
    mo --saved_model_dir /content/saved_model \
       --output_dir /content/openvino_model \
       --input_shape "[1,224,224,3]" \
       --data_type FP16
""")
print("Model converted and saved as .xml and .bin in '/content/openvino_model'.")


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
1305/1305 ━━━━━━━━━━━━━━━━━━━━ 291s 185ms/step - loss: 0.7787 - val_loss: 0.8438
Converting the model to OpenVINO format...
Model converted and saved as .xml and .bin in '/content/openvino_model'.


In [12]:
print("Converting the model to OpenVINO format...")
os.system(f"""
    mo --saved_model_dir /content/saved_model \
       --output_dir /content/xmll \
       --input_shape "[1,224,224,3]" \
       --data_type FP16
""")
print("Model converted and saved as .xml and .bin in '/content/xmll'.")


Converting the model to OpenVINO format...
Model converted and saved as .xml and .bin in '/content/xmll'.
